In [65]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px  #for map plot
import json #for Geojson data

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-RTL5926;'
                      'Database=AdventureWorks2019;'
                      'Trusted_Connection=yes;')    
                      


In [103]:
 #set a default place for fig to be shown without making it so heavy for Jupiter Note
import plotly.io as pio
pio.renderers.default='browser'

In [ ]:
#check the top performance country
Top_performing_country=pd.read_sql_query('SELECT * from top_performing_country',conn)
print(Top_performing_country)

In [ ]:
#create a df of regional sales in the US
region_sales_US=pd.read_sql_query('SELECT * from region_sales_US',conn)
print(region_sales_US)

In [ ]:
#plot the double bar chart of regional sales performance YTD and Last Year
labels = region_sales_US['region']
Sales_YTD = region_sales_US['Sales_YTD']
Sales_LastYear=region_sales_US['Sales_LastYear']

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, Sales_YTD, width, label='Sales_YTD',color='purple')
rects2 = ax.bar(x + width/2, Sales_LastYear, width, label='Sales_LastYear',color='orange')

# Add some text for labels, title and custom x-axis tick labels, etc.
#ax.set_xlabel('Regions')
ax.set_ylabel('Sales')
ax.set_title('Regional Sales YTD/Last Year US')
ax.set_xticks(x, labels)
ax.legend()

fig.tight_layout()

plt.show()

In [101]:
#the double pie chart have provided information about the diferences earch region
#but it has not show clearly the changes in percentage and total sales each period
#the below codes are for making a double pie chart to provide the missing information
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#set the labels for the donuts
labels = region_sales_US['region']

fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]])
                    #subplot_titles=['YTD', 'Last Year'])
#create the 2 pies
fig.add_trace(go.Pie(labels=labels, values=region_sales_US['Sales_YTD'], scalegroup='one',
                     name="Sales YTD"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=region_sales_US['Sales_LastYear'], scalegroup='one',
                     name="Sales Last Year"), 1, 2)
#make holes for donut charts
fig.update_traces(hole=.4, hoverinfo="label+percent+name")
#update layout with title and donut titles
fig.update_layout(
    title_text='Regional Sales YTD/ Last Year US',
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='YTD', x=0.20, y=0.5, font_size=20, showarrow=False),
                 dict(text='Last Year', x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.show()

In [ ]:
#the pie chart and bar chart show some differences about the sales YTD and Last Year.
#clearly, sales YTD is significantly higher that one Last year
# let's find out in which period the sale value is collected
latest_order_time=pd.read_sql_query('select max(OrderDate) from sales.SalesOrderHeader',conn)
print(latest_order_time)
#surprisingly, the YTD period is the first 6month of 2014

In [ ]:
#Stated to map the regions

In [ ]:

#read Geojson file about US geography
US_states = json.load(open('D:/2022/GENERATION/PROJECT_1/gz_2010_us_040_00_20m.json','r')) 
US_states['features'][1] #to check the value of geojson file, noticing the ID is the 'STATE' feature

In [ ]:
#create a dict including names and ids of states
state_id_map={}     
for feature in US_states['features']:
    feature['id']=feature['properties']['STATE']
    state_id_map[feature['properties']['NAME']]=feature['id']
print(state_id_map) 

In [ ]:
US_sales = pd.read_excel('D:/2022/GENERATION/PROJECT_1/US-states_sales.xlsx')  #read states_sales excel file
US_sales['State']=US_sales['State'].str.strip() #trim unwanted spaces, otherwise, it can't be match with state_id_map
print(US_sales)

In [ ]:
#add the id to link the sales tata with the geojson data using lambda x
US_sales['id'] = US_sales['State'].apply(lambda x: state_id_map[x])
print(US_sales)

In [89]:
#plot the map comparing the regional sales value YTD, result will be shown in browser
px.choropleth(US_sales,locations='id', 
                geojson=US_states,
                color='Sales_YTD',
                scope='usa',
                hover_name='State',
                hover_data=['Sales_YTD'],
                range_color=(2000000,1.1e+07), #this is to keep the color scale the same for the 2 periods
                title='US Regional Sales YTD')


In [90]:
#plot the map comparing the regional sales value last year, result will be shown in browser
px.choropleth(US_sales,locations='id', 
                geojson=US_states,
                color='Sales_LastYear',
                scope='usa',
                hover_name='State',
                range_color=(2000000,1.1e+07), #this is to keep the color scale the same for the 2 periods
                hover_data=['Sales_LastYear'],
                title='US Regional Sales Last Year')